In [ ]:
# import packages
import requests
from dotenv import load_dotenv
import os 
import pandas as pd

In [ ]:
# find and load .env file
load_dotenv()

In [ ]:
# get pushover app token and user from environment file 
PUSHOVER_TOKEN = os.getenv("PUSHOVER_TOKEN")
PUSHOVER_USER = os.getenv("PUSHOVER_USER")

# define function to send push notifications using Pushover app 
def send_push(message):
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": PUSHOVER_TOKEN,
            "user": PUSHOVER_USER,
            "message": message
        }
    )

In [ ]:
# import schedule with event IDs for team of interest (here, UConn)
from game_schedule import GAME_SCHEDULE

In [ ]:
# get in-game play-by-play data from ESPN for games in GAME_SCHEDULE

event_id = os.getenv('EVENT_ID') 
#event_id = '401825613'

# pull event id into API URL
espn_api_url = 'https://site.web.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/summary?region=us&lang=en&contentorigin=espn&event={event_id}'

# set parameters 
params = {
    'region': 'us',
    'lang': 'en',
    'contentorigin': 'espn',
    'event': event_id
}
# get data and format as json 
response = requests.get(espn_api_url, params=params)
data = response.json()

# print data 
# print(data)

{'boxscore': {'teams': [{'team': {'id': '77', 'uid': 's:40~l:54~t:77', 'slug': 'northwestern-wildcats', 'location': 'Northwestern', 'name': 'Wildcats', 'abbreviation': 'NU', 'displayName': 'Northwestern Wildcats', 'shortDisplayName': 'Northwestern', 'color': '582c83', 'alternateColor': 'ffffff', 'logo': 'https://a.espncdn.com/i/teamlogos/ncaa/500/77.png'}, 'statistics': [{'name': 'fieldGoalsMade-fieldGoalsAttempted', 'displayValue': '13-41', 'label': 'FG'}, {'name': 'fieldGoalPct', 'displayValue': '32', 'abbreviation': 'FG%', 'label': 'Field Goal %'}, {'name': 'threePointFieldGoalsMade-threePointFieldGoalsAttempted', 'displayValue': '1-7', 'label': '3PT'}, {'name': 'threePointFieldGoalPct', 'displayValue': '14', 'abbreviation': '3P%', 'label': 'Three Point %'}, {'name': 'freeThrowsMade-freeThrowsAttempted', 'displayValue': '6-9', 'label': 'FT'}, {'name': 'freeThrowPct', 'displayValue': '67', 'abbreviation': 'FT%', 'label': 'Free Throw %'}, {'name': 'totalRebounds', 'displayValue': '24'

In [ ]:
# pull subs for player of interest based on name or athlete ID 
# or team of interest
# add to env file
#player_name = 'KK Arnold'
team = 'UConn'

In [ ]:
sub_in = []

# get plays data 
# plays are stored under "plays" inside "header" or "gamecast" depending on the event
plays = data.get("plays", [])

# for each play, extract any player substitution data
for play in plays:
    play_type = play.get("type", {}).get("text", "")
    description = play.get("text", "")
    
    if play_type == "Substitution" and "subbing in" in description and team in description: #and player_name in description if want to limit to certain player(s)
        sub_in.append({
            "period": play.get("period", {}).get("displayValue"),
            "clock": play.get("clock", {}).get("displayValue"),
            "team_id": play.get("team", {}).get("id"),
            "description": play.get("text"),
            "athlete_ids": [p["athlete"]["id"] for p in play.get("participants", [])]
        })

# Print results
for sub in sub_in:
    print(sub)

{'period': '1st Quarter', 'clock': '6:43', 'team_id': '45', 'description': 'Mia James subbing in for George Washington', 'athlete_ids': ['5312781']}
{'period': '1st Quarter', 'clock': '6:38', 'team_id': '77', 'description': 'Angelina Hodgens subbing in for Northwestern', 'athlete_ids': ['5311594']}
{'period': '1st Quarter', 'clock': '5:37', 'team_id': '45', 'description': 'Emma Theodorsson subbing in for George Washington', 'athlete_ids': ['5109199']}
{'period': '1st Quarter', 'clock': '4:55', 'team_id': '77', 'description': 'Tate Lash subbing in for Northwestern', 'athlete_ids': ['4704252']}
{'period': '1st Quarter', 'clock': '4:55', 'team_id': '77', 'description': 'Kat Righeimer subbing in for Northwestern', 'athlete_ids': ['5240200']}
{'period': '1st Quarter', 'clock': '3:22', 'team_id': '45', 'description': 'Colleen Phiri subbing in for George Washington', 'athlete_ids': ['5312782']}
{'period': '1st Quarter', 'clock': '2:06', 'team_id': '45', 'description': 'Jaeda Wilson subbing in

In [7]:
# create data frame of substitutions 

subs = pd.DataFrame(sub_in)
subs.head(10)

,period,clock,team_id,description,athlete_ids
0,1st Quarter,6:43,45,Mia James subbing in for George Washington,[5312781]
1,1st Quarter,6:38,77,Angelina Hodgens subbing in for Northwestern,[5311594]
2,1st Quarter,5:37,45,Emma Theodorsson subbing in for George Washington,[5109199]
3,1st Quarter,4:55,77,Tate Lash subbing in for Northwestern,[4704252]
4,1st Quarter,4:55,77,Kat Righeimer subbing in for Northwestern,[5240200]
5,1st Quarter,3:22,45,Colleen Phiri subbing in for George Washington,[5312782]
6,1st Quarter,2:06,45,Jaeda Wilson subbing in for George Washington,[5240230]
7,1st Quarter,2:06,45,Filipa Calisto subbing in for George Washington,[4904644]
8,1st Quarter,2:06,45,Caia Loving subbing in for George Washington,[5107472]
9,1st Quarter,2:06,77,Xamiya Walton subbing in for Northwestern,[5240198]


In [21]:
# load existing substitutions
SEEN_FILE = "seen_subs.json"

if os.path.exists(SEEN_FILE):
    with open(SEEN_FILE, "r") as f:
        seen = set(json.load(f))
else:
    seen = set()

# get subs
sub_in = []
plays = data.get("plays", [])

for play in plays:
    play_type = play.get("type", {}).get("text", "")
    description = play.get("text", "")
    seq = play.get("sequenceNumber")
    period = play.get("period", {}).get("displayValue")

    if play_type == "Substitution" and "subbing in" in description.lower() and period == "3rd Quarter":
        sub_in.append({
            "sequenceNumber": seq,
            "period": play.get("period", {}).get("displayValue"),
            "clock": play.get("clock", {}).get("displayValue"),
            "team_id": play.get("team", {}).get("id"),
            "description": description,
            "athlete_ids": [p["athlete"]["id"] for p in play.get("participants", [])]
        })

# identify new subs
new_subs = [s for s in sub_in if s["sequenceNumber"] not in seen]
print(new_subs)

[{'sequenceNumber': '116252839', 'period': '3rd Quarter', 'clock': '10:00', 'team_id': '45', 'description': 'Jaeda Wilson subbing in for George Washington', 'athlete_ids': ['5240230']}, {'sequenceNumber': '116252842', 'period': '3rd Quarter', 'clock': '10:00', 'team_id': '45', 'description': 'Sara Lewis subbing in for George Washington', 'athlete_ids': ['5174364']}, {'sequenceNumber': '116252846', 'period': '3rd Quarter', 'clock': '10:00', 'team_id': '77', 'description': 'Caroline Lau subbing in for Northwestern', 'athlete_ids': ['4565529']}, {'sequenceNumber': '116252847', 'period': '3rd Quarter', 'clock': '10:00', 'team_id': '77', 'description': 'Tayla Thomas subbing in for Northwestern', 'athlete_ids': ['5240201']}, {'sequenceNumber': '116253229', 'period': '3rd Quarter', 'clock': '7:14', 'team_id': '45', 'description': 'Emma Theodorsson subbing in for George Washington', 'athlete_ids': ['5109199']}, {'sequenceNumber': '116253230', 'period': '3rd Quarter', 'clock': '7:14', 'team_id'

In [24]:
# send text about any new subs 
for sub in new_subs:
    message_body = (
        f"NEW SUB: {sub['description']} "
        f"({sub['period']} @ {sub['clock']})"
    )

    send_push(message_body)

# update previously seen subs 
for sub in new_subs:
    seen.add(sub["sequenceNumber"])

with open(SEEN_FILE, "w") as f:
    json.dump(list(seen), f)